In [0]:
stg_matches_df = spark.table('default.stg_matches')
stg_teams_df = spark.table('default.stg_teams')

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

def matches_teams_OBT(stg_matches_df: DataFrame, stg_teams_df: DataFrame) -> DataFrame:
    home_cols_prefixed = {c: 'home_' + c for c in stg_teams_df.columns}
    marts_home_teams_df = (
        stg_matches_df
        .join(
            other=stg_teams_df
                .withColumnsRenamed(home_cols_prefixed),
            on=['home_team_id'],
            how='left'
        )
    )

    away_cols_prefixed = {c: 'away_' + c for c in stg_teams_df.columns}
    marts_all_teams_df = (
        marts_home_teams_df
        .join(
            other=stg_teams_df
                .withColumnsRenamed(away_cols_prefixed),
            on=['away_team_id'],
            how='left'
        )
    )

    return marts_all_teams_df

marts_matches_teams_df = matches_teams_OBT(stg_matches_df, stg_teams_df)
display(marts_matches_teams_df.limit(3))


In [0]:
def replace_match_winners(marts_matches_teams_df: DataFrame) -> DataFrame:
    return (
        marts_matches_teams_df.withColumn('winner_name',
            F.when(marts_matches_teams_df.winner == 'HOME_TEAM', marts_matches_teams_df.home_team_name) \
            .when(marts_matches_teams_df.winner == 'AWAY_TEAM', marts_matches_teams_df.home_team_name) \
            .otherwise(None))
    )

marts_matches_teams_df = replace_match_winners(marts_matches_teams_df)
display(marts_matches_teams_df.limit(3))

In [0]:
marts_matches_teams_df.write.mode('overwrite').saveAsTable('marts_matches_teams')